# Masking a range

In this example, we will use a slider to mask data outside a selected range in a two-dimensional image.
The data inside the range will be summed along the vertical dimension,
and displayed as a one-dimensional plot below the image.

In [ ]:
%matplotlib widget
import plopp as pp
import scipp as sc
import ipywidgets as ipw

We first generate some data that contains three bands of peaks that all have different spreads.

In [ ]:
from plopp.data.examples import three_bands

da = three_bands()

We then construct our interface with a slider, a node that adds a mask, and a node that sums the unmasked data along the `y` dimension.

In [ ]:
ydim = 'y'


@pp.node
def add_mask(da, trunc_range):
    """A function that will add a mask outside of selected range."""
    min_tr, max_tr = trunc_range
    out = da.copy(deep=False)
    out.masks[ydim] = sc.zeros(sizes={ydim: da.sizes[ydim]}, dtype=bool)
    out.masks[ydim].values[:min_tr] = True
    out.masks[ydim].values[max_tr:] = True
    return out


# Make a range slider
slider = ipw.IntRangeSlider(
    max=da.sizes[ydim] - 1, description='y indices', layout={'width': '400px'}
)
slider_node = pp.widget_node(slider)

# Add mask
mask_node = add_mask(da, trunc_range=slider_node)

# Node that sums along the y dimension
sum_node = pp.Node(sc.sum, mask_node, dim=ydim)

# Make figures
image = pp.imagefigure(mask_node, norm='log')
lines = pp.linefigure(sum_node, grid=True, ls='solid', marker='')
ipw.VBox([slider, image, lines])

In [ ]:
pp.show_graph(lines)

In [ ]:
# This cell is used to generate the thumbnail for the docs gallery.
# It is hidden from the online documentation.
image.save('../_static/gallery/masking-a-range-thumbnail.png')